# Human Detection Project

In [211]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.transform import pyramid_gaussian

# Lectura de imágenes

In [5]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

### Fotos Tain 

In [6]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

### Fotos Test

In [261]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotosTest = leerFotos(path)

### Recorte Train Neg

In [7]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroCortesxPh = 20      # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [8]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

# Extracción de Caracterísitcas
    L1
       Normalization using L1-norm.
    L1-sqrt
       Normalization using L1-norm, followed by square root.
    L2
       Normalization using L2-norm.
    L2-Hys
       Normalization using L2-norm, followed by limiting the
       maximum values to 0.2 (`Hys` stands for `hysteresis`) and
       renormalization using L2-norm. (default)

In [249]:
orientations    = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm      = 'L2' 
visualize       = False
visualise       = None
transform_sqrt  = True
feature_vector  = True

In [251]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector))

    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [189]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

In [190]:
fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [191]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

print(fotosTrain.shape)

(4563, 3780)


# Entrenamiento del modelo

In [210]:
clf = svm.LinearSVC(C = 0.007)
clf.fit(fotosTrain,y.ravel())

LinearSVC(C=0.007, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [209]:
clf.score(fdPosTrain,yPos)

0.9769094138543517

# Slide Window

In [255]:
winSize = 134,70

In [241]:
def slidWindow(imagen,maskHeight = 134,maskWidth = 70,step = 10):
    arrayImagen = []
    for i in range(0,imagen.shape[0]-maskHeight,step):        
        for j in range(0,imagen.shape[1]-maskWidth,step):
            arrayImagen.append((i,j,imagen[i:i+maskHeight, j:j+maskWidth]))
    return arrayImagen

# Pyramid Gaussian

In [258]:
def estudiaFoto(im,winSize,clf,ratio = 2):#Ratio de reducción
    # List to store the detections
    hdetections = []

    for scaLev,imScaled in enumerate(pyramid_gaussian(im,downscale=ratio)):

        if (imScaled.shape[0] < winSize[0] or imScaled.shape[1] < winSize[1]):
            break
            
        for (x,y,imVentana) in slidWindow(imScaled,step=5):
            fdSc = hog(imVentana,orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
            pred = clf.predict(fdSc.reshape(1,-1))
            if (pred == 1):
                hdetections.append((x,y,int(winSize[0]*ratio**scaLev),int(winSize[1]*ratio**scaLev)))
    return hdetections

# Prueba del Modelo

In [266]:
def muestraResultado(detections,im):
    for (x_tl, y_tl,w, h) in detections:
            # Draw the detections
            cv2.rectangle(im, (x_tl, y_tl), (x_tl+w, y_tl+h), (0, 0, 0), thickness=2)
    cv2.imshow("Raw Detections before NMS", im)
    cv2.waitKey(0)

In [265]:
muestraResultado(humanDetec,fotosTest[6])

In [254]:
%%time
humanDetec = estudiaFoto(fotosTest[6],winSize,clf)

C:\Users\Jhonny Chicaiza\Anaconda3\envs\py37vision\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


Wall time: 38.1 s


C:\Users\Jhonny Chicaiza\Anaconda3\envs\py37vision\lib\site-packages\skimage\transform\_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


In [253]:
%%time
listOk = []
partImagen = slidWindow(fotostest[6])
for i in range(len(partImagen)):
    pruebaHog = hog(partImagen[i][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
    if (1 == clf.predict(pruebaHog.reshape(1,-1))):
        listOk.append(i);

Wall time: 7.88 s


In [182]:
cv2.imshow('img ', fotostest[6])
cv2.imshow('xd ', partImagen[2291])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [221]:
print(len(listOk))
print(listOk)

48
[113, 238, 437, 503, 836, 856, 925, 926, 1014, 1104, 1159, 1160, 1220, 1221, 1281, 1351, 1557, 1571, 1581, 1585, 1620, 1627, 1628, 1629, 1631, 1632, 1651, 1670, 1701, 1721, 1726, 1771, 1808, 1841, 1856, 1857, 1908, 1944, 1978, 2081, 2120, 2151, 2190, 2197, 2221, 2291, 2557, 2617]


In [229]:
 pruebaHog = hog(partImagen[2291][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector,multichannel);
clf.decision_function(pruebaHog.reshape(1,-1))

array([0.20889192])